# VGG16을 사용하여 아이돌 이미지 분류
- 이미지는 Kaggle에서 다운로드 가능(https://www.kaggle.com/datasets/vkehfdl1/kidf-kpop-idol-dataset-female?resource=download&select=kid_f_train.csv)


In [71]:
import numpy as np
import pandas as pd
import os

from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

from sklearn.preprocessing import LabelEncoder

In [155]:
url = './dataset'

csv_url_train = url + '/kid_f_train.csv'
csv_url_test = url + '/kid_f_test.csv'


train_labels = pd.read_csv(csv_url_train)
test_labels = pd.read_csv(csv_url_test)

In [158]:
# 라벨값 비율 확인
print('훈련 데이터셋 shape > ', train_labels.shape)
print('훈련 데이터셋 비율 top 5')
print((train_labels.groupby('name').size() / train_labels.shape[0] * 100).sort_values(ascending=False).head(5))

print('\n테스트 데이터셋 shape > ', test_labels.shape)
print('테스트 데이터셋 비율 top 5')
print((test_labels.groupby('name').size() / test_labels.shape[0] * 100).sort_values(ascending=False).head(5))

훈련 데이터셋 shape >  (5591, 2)
훈련 데이터셋 비율 top 5
name
lisa        22.929708
rose        22.267931
jisoo       18.297263
jennie       7.404758
kimminju     4.668217
dtype: float64

테스트 데이터셋 shape >  (300, 2)
테스트 데이터셋 비율 top 5
name
jisoo     24.000000
iu        21.333333
rose      14.000000
lisa      13.333333
jennie     5.000000
dtype: float64


# 데이터 전처리
- 훈련 데이터셋 중 lisa, rose, jisoo의 비율이 절반 이상이 넘음 => 따라서 세 명은 훈련 데이터, 학습 데이터에서 삭제
- 테스트 데이터셋에는 iu, chaeyoungdl 있지만, 테스트 데이터셋에는 없음 => 두 명도 삭제
- 텐서화 
 - 세명의 데이터를 삭제한 데이프레임을 기준으로 이미지 불러와서 텐서화
 - 이미지파일만 있는 것이 아니라 
 Thumbs.db' 파일도 있으므로 텐서로 변활할 때 에러 발생이 예상되므로 예외처리 하기


In [159]:
# 리사, 로제, 지수 제거
drop_name = ['lisa', 'rose', 'jisoo', 'iu', 'chaeyoung']

train_drop_index = train_labels[train_labels['name'].isin(drop_name)].index
test_drop_index = test_labels[test_labels['name'].isin(drop_name)].index

train_labels = train_labels.drop(train_drop_index)
test_labels = test_labels.drop(test_drop_index)

In [162]:
# 라벨 인코딩
encoder = LabelEncoder()
encoder.fit(train_labels['name'])

train_encoded = encoder.transform(train_labels['name'])
test_encoded = encoder.transform(test_labels['name'])

train_labels['name'] = train_encoded
test_labels['name'] = test_encoded

In [165]:
print("###### 세 명 삭제 후 ######")
print("train_labels 데이터 개수 : ", len(train_labels))
print("train_labels 클래스 개수 : ", len(train_labels['name'].unique()))
print('----------' * 3)
print("test_labels 데이터 개수 : ", len(test_labels))
print("test_labels클래스 개수 : ", len(test_labels['name'].unique()))

###### 세 명 삭제 후 ######
train_labels 데이터 개수 :  2041
train_labels 클래스 개수 :  52
------------------------------
test_labels 데이터 개수 :  81
test_labels클래스 개수 :  20


In [150]:
def image_to_tensor(df, dir_url):
    file_list = os.listdir(dir_url)
    result_list = []

    for i in tqdm(range(df.shape[0])):
        try:
            file_name = df['file_name'].iloc[i] # 파일 이름 Ex)100.jpg
            label = df['name'].iloc[i] # 연예인 이름

            # 이미지 오픈 및 텐서화
            file_url = dir_url + '/' + file_name
            img = Image.open(file_url)
            tf = transforms.Totensor()
            
            tensor_img = tf(img)
            # 튜플 형태로 저장
            result_list.append((tensor_img, label))
        except:
            print("파일명 > ", file_name, "\n연예인명 > ", label)
            
    return result_list

In [152]:
train_dir_url = './dataset/HQ_512x512/HQ_512x512' 
test_dir_url = './dataset/test_final_with_degrad/test'

train_dataset = image_to_tensor(train_labels, train_dir_url)
print('*'*100)
test_dataset = image_to_tensor(test_labels, test_dir_url)

 16%|█▌        | 318/2041 [00:00<00:01, 1537.24it/s]

파일명 >  5.jpg 
연예인명 >  51
파일명 >  8.jpg 
연예인명 >  51
파일명 >  17.jpg 
연예인명 >  51
파일명 >  23.jpg 
연예인명 >  51
파일명 >  26.jpg 
연예인명 >  51
파일명 >  29.jpg 
연예인명 >  51
파일명 >  30.jpg 
연예인명 >  51
파일명 >  34.jpg 
연예인명 >  51
파일명 >  35.jpg 
연예인명 >  51
파일명 >  38.jpg 
연예인명 >  51
파일명 >  39.jpg 
연예인명 >  51
파일명 >  41.jpg 
연예인명 >  51
파일명 >  51.jpg 
연예인명 >  51
파일명 >  52.jpg 
연예인명 >  51
파일명 >  53.jpg 
연예인명 >  51
파일명 >  54.jpg 
연예인명 >  51
파일명 >  55.jpg 
연예인명 >  51
파일명 >  56.jpg 
연예인명 >  51
파일명 >  57.jpg 
연예인명 >  51
파일명 >  60.jpg 
연예인명 >  51
파일명 >  63.jpg 
연예인명 >  51
파일명 >  64.jpg 
연예인명 >  51
파일명 >  65.jpg 
연예인명 >  51
파일명 >  71.jpg 
연예인명 >  51
파일명 >  72.jpg 
연예인명 >  51
파일명 >  75.jpg 
연예인명 >  51
파일명 >  77.jpg 
연예인명 >  51
파일명 >  78.jpg 
연예인명 >  51
파일명 >  80.jpg 
연예인명 >  51
파일명 >  81.jpg 
연예인명 >  51
파일명 >  82.jpg 
연예인명 >  51
파일명 >  83.jpg 
연예인명 >  51
파일명 >  84.jpg 
연예인명 >  51
파일명 >  85.jpg 
연예인명 >  51
파일명 >  87.jpg 
연예인명 >  51
파일명 >  89.jpg 
연예인명 >  51
파일명 >  90.jpg 
연예인명 >  51
파일명 >  93.jpg 
연예인명 >  51
파일명 >  94.jpg 

 23%|██▎       | 474/2041 [00:00<00:01, 1529.66it/s]

파일명 >  1076.jpg 
연예인명 >  27
파일명 >  1077.jpg 
연예인명 >  27
파일명 >  1078.jpg 
연예인명 >  27
파일명 >  1080.jpg 
연예인명 >  27
파일명 >  1083.jpg 
연예인명 >  27
파일명 >  1084.jpg 
연예인명 >  27
파일명 >  1085.jpg 
연예인명 >  27
파일명 >  1086.jpg 
연예인명 >  27
파일명 >  1087.jpg 
연예인명 >  27
파일명 >  1088.jpg 
연예인명 >  27
파일명 >  1092.jpg 
연예인명 >  27
파일명 >  1094.jpg 
연예인명 >  27
파일명 >  1095.jpg 
연예인명 >  27
파일명 >  1101.jpg 
연예인명 >  27
파일명 >  1102.jpg 
연예인명 >  27
파일명 >  1103.jpg 
연예인명 >  27
파일명 >  1104.jpg 
연예인명 >  27
파일명 >  1116.jpg 
연예인명 >  27
파일명 >  1117.jpg 
연예인명 >  27
파일명 >  1118.jpg 
연예인명 >  27
파일명 >  1119.jpg 
연예인명 >  27
파일명 >  1129.jpg 
연예인명 >  27
파일명 >  1130.jpg 
연예인명 >  27
파일명 >  1131.jpg 
연예인명 >  27
파일명 >  1138.jpg 
연예인명 >  27
파일명 >  1139.jpg 
연예인명 >  27
파일명 >  1143.jpg 
연예인명 >  27
파일명 >  1145.jpg 
연예인명 >  27
파일명 >  1146.jpg 
연예인명 >  27
파일명 >  1160.jpg 
연예인명 >  27
파일명 >  1161.jpg 
연예인명 >  27
파일명 >  1162.jpg 
연예인명 >  27
파일명 >  1177.jpg 
연예인명 >  27
파일명 >  1179.jpg 
연예인명 >  27
파일명 >  1180.jpg 
연예인명 >  27
파일명 >  1181.jpg 
연예인

 39%|███▉      | 791/2041 [00:00<00:00, 1526.94it/s]

...

파일명 >  50865.jpg 
연예인명 >  17
파일명 >  50867.jpg 
연예인명 >  17
파일명 >  50868.jpg 
연예인명 >  17
파일명 >  50869.jpg 
연예인명 >  17
파일명 >  50887.jpg 
연예인명 >  17
파일명 >  50895.jpg 
연예인명 >  17
파일명 >  50923.jpg 
연예인명 >  17
파일명 >  50948.jpg 
연예인명 >  17
파일명 >  50955.jpg 
연예인명 >  17
파일명 >  50965.jpg 
연예인명 >  17
파일명 >  50980.jpg 
연예인명 >  17
파일명 >  50988.jpg 
연예인명 >  17
파일명 >  51133.jpg 
연예인명 >  17
파일명 >  51146.jpg 
연예인명 >  17
파일명 >  51200.jpg 
연예인명 >  17
파일명 >  51440.jpg 
연예인명 >  17
파일명 >  51762.jpg 
연예인명 >  17
파일명 >  51767.jpg 
연예인명 >  17
파일명 >  51786.jpg 
연예인명 >  17
파일명 >  51847.jpg 
연예인명 >  17
파일명 >  51888.jpg 
연예인명 >  17
파일명 >  51902.jpg 
연예인명 >  17
파일명 >  52015.jpg 
연예인명 >  17
파일명 >  52039.jpg 
연예인명 >  17
파일명 >  52056.jpg 
연예인명 >  17
파일명 >  52059.jpg 
연예인명 >  17
파일명 >  52169.jpg 
연예인명 >  17
파일명 >  52174.jpg 
연예인명 >  17
파일명 >  52209.jpg 
연예인명 >  17
파일명 >  52214.jpg 
연예인명 >  17
파일명 >  52216.jpg 
연예인명 >  17
파일명 >  52300.jpg 
연예인명 >  17
파일명 >  52303.jpg 
연예인명 >  17
파일명 >  52304.jpg 
연예인명 >  17
파일명 >  52306.j

100%|██████████| 2041/2041 [00:01<00:00, 1419.68it/s]


****************************************************************************************************


100%|██████████| 81/81 [00:00<00:00, 1187.87it/s]

파일명 >  45.jpg 
연예인명 >  51
파일명 >  85.jpg 
연예인명 >  46
파일명 >  96.jpg 
연예인명 >  46
파일명 >  112.jpg 
연예인명 >  46
파일명 >  116.jpg 
연예인명 >  51
파일명 >  127.jpg 
연예인명 >  51
파일명 >  201.jpg 
연예인명 >  51
파일명 >  252.jpg 
연예인명 >  46
파일명 >  299.jpg 
연예인명 >  47
파일명 >  312.jpg 
연예인명 >  47
파일명 >  351.jpg 
연예인명 >  47
파일명 >  425.jpg 
연예인명 >  6
파일명 >  435.jpg 
연예인명 >  6
파일명 >  448.jpg 
연예인명 >  6
파일명 >  522.jpg 
연예인명 >  6
파일명 >  610.jpg 
연예인명 >  6
파일명 >  835.jpg 
연예인명 >  6
파일명 >  884.jpg 
연예인명 >  6
파일명 >  910.jpg 
연예인명 >  27
파일명 >  1005.jpg 
연예인명 >  27
파일명 >  1059.jpg 
연예인명 >  27
파일명 >  1144.jpg 
연예인명 >  27
파일명 >  1324.jpg 
연예인명 >  27
파일명 >  1330.jpg 
연예인명 >  27
파일명 >  1342.jpg 
연예인명 >  27
파일명 >  1505.jpg 
연예인명 >  27
파일명 >  1578.jpg 
연예인명 >  27
파일명 >  1597.jpg 
연예인명 >  27
파일명 >  1721.jpg 
연예인명 >  22
파일명 >  1782.jpg 
연예인명 >  22
파일명 >  1792 .jpg 
연예인명 >  22
파일명 >  1899.jpg 
연예인명 >  22
파일명 >  2022.jpg 
연예인명 >  22
파일명 >  2111.jpg 
연예인명 >  22
파일명 >  2126.jpg 
연예인명 >  22
파일명 >  2168.jpg 
연예인명 >  22
파일명 >  2625.jpg 
연예인

In [153]:
print("train_dataset 데이터 개수 : ", len(train_dataset))

lst = []
for i in range(len(train_dataset)):
    lst.append(train_dataset[i][1])
s_lst = pd.Series(lst)
print('train_dataset 라벨 개수', len(s_lst.unique()))

print('----------' * 3)

print("test_dataset 데이터 개수 : ", len(test_dataset))

lst = []
for i in range(len(test_dataset)):
    lst.append(test_dataset[i][1])
s_lst = pd.Series(lst)
print('test_dataset 라벨 개수', len(s_lst.unique()))

train_dataset 데이터 개수 :  0
train_dataset 라벨 개수 0
------------------------------
test_dataset 데이터 개수 :  0
test_dataset 라벨 개수 0


C:\Users\rlawl\AppData\Local\Temp\ipykernel_10452\1731903526.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s_lst = pd.Series(lst)
C:\Users\rlawl\AppData\Local\Temp\ipykernel_10452\1731903526.py:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s_lst = pd.Series(lst)


In [154]:
train_dataset[0][0].shape

IndexError: list index out of range

In [ ]:
# 데이터 사이즈를 512x512에서 224x224로 변경


# 모델링

In [111]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [112]:
# DataLoader 만들기
batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_laoder = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [113]:
# VGG 모델은 여러가지 타입이 있으므로 타입에 맞춰 모델 생성하기 위해 layer를 거친 후 출력 kernel의 개수 미리 지정(M은 pooling layer)
cfg = {
    'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

# A : VGG-11
# B : VGG-13
# D : VGG-16
# E : VGG-19

def make_layer(config): # 위에서 정의한 타입별로 모델 생성
    layers = []
    in_planes = 3 # input 개수
    for value in config:
        if value == "M": # Pooling layer일 때 
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else: # Conv  layer일 때
            layers.append(nn.Conv2d(in_planes, value, kernel_size=3, padding=1))
            layers.append(nn.ReLU())
            in_planes = value
    return nn.Sequential(*layers)

In [114]:
class VGG(nn.Module):
    def __init__(self, config, num_classes=1000, cifar=False):
        super(VGG, self).__init__()
        self.features = make_layer(config)
        
        # ImageNet
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes)  
        )
        # CIFAR-10
        if cifar:
            self.classifier = nn.Sequential(
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(True),
                nn.Dropout(0.5),
                nn.Linear(512, 512),
                nn.ReLU(True),
                nn.Linear(512, 10)  
            ) 
        
    def forward(self, x):
        out = self.features(x)
        out = torch.flatten(out,1)
        out = self.classifier(out)
        return out

In [115]:
# 지정한 VGG 타입 별 모델을 생성하는 함수 만들기
def VGG11(cifar=False):
    return VGG(config = cfg['A'], cifar = cifar)

def VGG13(cifar=False):
    return VGG(config = cfg['B'], cifar = cifar)

def VGG16(cifar=False):
    return VGG(config = cfg['D'], cifar = cifar)

def VGG19(cifar=False):
    return VGG(config = cfg['E'], cifar = cifar)

In [116]:
model = VGG16()

In [121]:
from torchsummary import summary

summary(model=model,
        input_size=(3, 224, 224),
        batch_size=batch_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [32, 64, 224, 224]           1,792
              ReLU-2         [32, 64, 224, 224]               0
            Conv2d-3         [32, 64, 224, 224]          36,928
              ReLU-4         [32, 64, 224, 224]               0
         MaxPool2d-5         [32, 64, 112, 112]               0
            Conv2d-6        [32, 128, 112, 112]          73,856
              ReLU-7        [32, 128, 112, 112]               0
            Conv2d-8        [32, 128, 112, 112]         147,584
              ReLU-9        [32, 128, 112, 112]               0
        MaxPool2d-10          [32, 128, 56, 56]               0
           Conv2d-11          [32, 256, 56, 56]         295,168
             ReLU-12          [32, 256, 56, 56]               0
           Conv2d-13          [32, 256, 56, 56]         590,080
             ReLU-14          [32, 256,